In [1]:
from xpbd_softbody import XPBDSoftbody
import pyvista as pv
import config as cfg
import matplotlib.pyplot as plt
import torch
from xpbd_softbody_layer import XPBDStep
import numpy as np
from sklearn.neighbors import KDTree
from xpbd_softbody_layer import get_energy_boundary
from scipy.interpolate import interp1d
from cubic_bezier import *
from tqdm import trange
from torchviz import make_dot
import os
from tqdm import trange

In [2]:
path='../assets'
softbody = XPBDSoftbody()
mesh = softbody.add_thinshell(pv.read(os.path.join(path, 'grape_skin.ply')), n_surf=cfg.n_surf)
mesh = softbody.add_thinshell(pv.read(os.path.join(path, 'grape_skin.ply')), n_surf=cfg.n_surf)
# mesh = softbody.add_thinshell(pv.Plane(), n_surf=cfg.n_surf)
softbody.init_states()
softbody.init_dist_constraints()

softbody.add_multi_boundary_constrain(1, 0, 0.0006, range(600))
softbody.fix_indice(0, range(600))

detect 2 object


In [3]:
softbody.C_boundary_list[0].shape

torch.Size([690, 2])

In [4]:
V_origin = softbody.V.clone()
V_velocity_origin = softbody.V_velocity.clone()

In [5]:
# stiffness setting
cloth_dist_stiffness = 1
V_boundary_stiffness = 0.1
V_dist_stiffness = torch.ones_like(softbody.V_mass).to(cfg.device)
V_boundary_stiffness = torch.ones((softbody.C_boundary_list[0].shape[0], 1)).to(cfg.device) * V_boundary_stiffness

In [6]:
V_boundary_stiffness.shape

torch.Size([690, 1])

In [7]:
u = torch.linspace(0, 1, 3).to(cfg.device)
us = torch.linspace(0, 1, 50).to(cfg.device)
start_point = np.array([[0.000224, 0.010794, -0.001233]])
start_point = torch.from_numpy(start_point).to(cfg.device)
# spline_control = np.array([ [0.000224, 0.010794, 0.003],
#                             [0.000224, 0.010794, 0.006],
#                             [0.000224, 0.010794, 0.009]])
spline_control = np.array([ [0.000224, 0.009794, 0.001],
                            [0.000224, 0.008794, 0.0020],
                            [0.000224, 0.007794, 0.0035]])
spline_control = torch.from_numpy(spline_control).to(cfg.device)

In [8]:
control_point = 31 + softbody.offset_list[1]
softbody.fix_point(1, 31)

In [9]:
V_boundary_stiffness.shape

torch.Size([690, 1])

In [10]:
pv.set_plot_theme('document')
pl = pv.Plotter()

# skin being peeled
pl.add_mesh(mesh, color='#9f5547ff', show_edges=True, edge_color='#b37164ff',  lighting=False,style='surface')
# ellipsoid grape
grape = pv.read('../assets/grape.ply')
grape.points = grape.points - np.array([0, 0, 1e-4])
pl.add_mesh(grape, color='#9f5547ff', show_edges=False, lighting=False,style='surface')

# grape meat that is green
grape_meat= pv.read('../assets/grape_skin.ply')
grape_meat.points = grape_meat.points - np.array([0, 0, 2e-4])
pl.add_mesh(grape_meat, color='#c0ab5eff', show_edges=False, lighting=False,style='surface')

Actor (0x7fe25a4fd7e0)
  Center:                     (-9.851530194282532e-06, 0.0014494163915514946, -0.0009795548299327494)
  Pickable:                   True
  Position:                   (0.0, 0.0, 0.0)
  Scale:                      (1.0, 1.0, 1.0)
  Visible:                    True
  X Bounds                    -6.636E-03, 6.616E-03
  Y Bounds                    -1.040E-02, 1.330E-02
  Z Bounds                    -3.189E-03, 1.230E-03
  User matrix:                Set
  Has mapper:                 True

Property (0x7fe25a4fdc00)
  Ambient:                     0.0
  Ambient color:               Color(hex='#c0ab5eff', opacity=255)
  Anisotropy:                  0.0
  Color:                       Color(hex='#c0ab5eff', opacity=255)
  Culling:                     "none"
  Diffuse:                     1.0
  Diffuse color:               Color(hex='#c0ab5eff', opacity=255)
  Edge color:                  Color(name='black', hex='#000000ff', opacity=255)
  Edge opacity:                1.0
 

In [11]:
# setup opt para
spline_control.requires_grad_(True)

tensor([[0.0002, 0.0098, 0.0010],
        [0.0002, 0.0088, 0.0020],
        [0.0002, 0.0078, 0.0035]], dtype=torch.float64, requires_grad=True)

In [12]:
optimizer = torch.optim.Adam([spline_control], lr = 0.001)

In [13]:
## point position target
# target = torch.tensor([0.01, 0.01, 0.01])
# target_point = 25 + softbody.offset_list[1]

In [14]:
## stiffness target
target = torch.ones_like(V_boundary_stiffness) * 1e-5

In [15]:
def loss_fn(target, pred):
    return torch.norm(target - pred)

In [16]:
# pl.open_gif('grape.gif')
for t in range(10):
    x_con = torch.cat((start_point[:, 0], spline_control[:, 0]))
    y_con = torch.cat((start_point[:, 1], spline_control[:, 1]))
    z_con = torch.cat((start_point[:, 2], spline_control[:, 2]))

    spline = cubic_bezier_arc_3D(x_con, y_con, z_con, 0.00025)
    print(spline.shape)
    spline_x = spline[0]
    spline_y = spline[1]
    spline_z = spline[2]

    spline_trajectory = torch.transpose(torch.vstack((spline_x, spline_y, spline_z)), 0, 1)
    # reset for each test
    softbody.V = V_origin.clone()
    softbody.V_velocity = V_velocity_origin.clone()

    cloth_dist_stiffness = 1
    V_boundary_stiffness = 0.1
    V_dist_stiffness = torch.ones_like(softbody.V_mass).to(cfg.device)
    V_boundary_stiffness = torch.ones((softbody.C_boundary_list[0].shape[0], 1)).to(cfg.device) * V_boundary_stiffness
    energy_total = torch.zeros(1).to(cfg.device)
    for i in trange(spline_trajectory.shape[0]):
        softbody.V[control_point] = spline_trajectory[i]
        step_ref = XPBDStep(softbody,
                        V_dist_stiffness=V_dist_stiffness, 
                        V_shape_stiffness=None,
                        V_boundary_stiffness=V_boundary_stiffness, 
                        dt=cfg.dt,
                        substep=cfg.substep,
                        iteration=cfg.iteration,
                        quasi_static=cfg.quasi_static,
                        plane_height=cfg.ground_plane_height, 
                        use_shape_matching=cfg.use_shape_matching,
                        use_spring_boundary=cfg.use_spring_boundary,
                        use_dist=True) #cfg.use_spring_boundary
        V_ref, V_velocity_ref = step_ref.forward(softbody.V, softbody.V_velocity)
        softbody.V = V_ref.clone()
        softbody.V_velocity = V_velocity_ref.clone()
        ref_V_boundary_stiffness = V_boundary_stiffness.clone()
        energy = get_energy_boundary(softbody, softbody.V, ref_V_boundary_stiffness)
        # set boundary energy that larger than 1e-8 as broken, 1e-5 is a eps to avoid divide 0
        # 1e-8 is the threshold, if energy is larger than 1e-8, the first term will be 0 and the last term will be 1e-5 * 1.
        V_boundary_stiffness = ref_V_boundary_stiffness * torch.sigmoid(1e10 * (1e-8 - energy)) + 1e-5*torch.sigmoid(1e10 * (energy - 1e-8))
        mesh.points = softbody.V[-600:].detach().cpu().numpy()
    
    #optimize
    loss = loss_fn(target, V_boundary_stiffness)
    loss.backward()
    print(spline_control.grad)
    optimizer.step()
    optimizer.zero_grad()
    # pl.write_frame()
# pl.close()


torch.Size([3, 23])


  0%|          | 0/23 [00:00<?, ?it/s]

100%|██████████| 23/23 [00:38<00:00,  1.69s/it]


tensor([[ 0.0122, -0.0118,  0.0161],
        [ 0.0244, -0.0202, -0.0193],
        [ 0.0051, -0.0138, -0.0202]], dtype=torch.float64)
torch.Size([3, 26])


 27%|██▋       | 7/26 [00:12<00:33,  1.75s/it]Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7fe354680e20>>
Traceback (most recent call last):
  File "/home/youcheng/anaconda3/envs/py_pbd/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 770, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 
 54%|█████▍    | 14/26 [00:25<00:21,  1.79s/it]

In [ ]:
# pl.open_gif('grape.gif')
# x_con = torch.cat((start_point[:, 0], spline_control[:, 0]))
# y_con = torch.cat((start_point[:, 1], spline_control[:, 1]))
# z_con = torch.cat((start_point[:, 2], spline_control[:, 2]))

# # spline_x = cubic_bezier(x_con, us)
# # spline_y = cubic_bezier(y_con, us)
# # spline_z = cubic_bezier(z_con, us)

# # spline_x = interp(u, x_con, us)
# # spline_y = interp(u, y_con, us)
# # spline_z = interp(u, z_con, us)

# spline = cubic_bezier_arc_3D(x_con, y_con, z_con, 0.00025)
# print(spline.shape)
# spline_x = spline[0]
# spline_y = spline[1]
# spline_z = spline[2]

# spline_trajectory = torch.transpose(torch.vstack((spline_x, spline_y, spline_z)), 0, 1)
# # reset for each test
# softbody.V = V_origin.clone()
# softbody.V_velocity = V_velocity_origin.clone()

# cloth_dist_stiffness = 1
# V_boundary_stiffness = 0.1
# V_dist_stiffness = torch.ones_like(softbody.V_mass).to(cfg.device)
# V_boundary_stiffness = torch.ones((softbody.C_boundary_list[0].shape[0], 1)).to(cfg.device) * V_boundary_stiffness
# energy_total = torch.zeros(1).to(cfg.device)
# for i in trange(spline_trajectory.shape[0]):
#     softbody.V[control_point] = spline_trajectory[i]
#     step_ref = XPBDStep(softbody,
#                     V_dist_stiffness=V_dist_stiffness, 
#                     V_shape_stiffness=None,
#                     V_boundary_stiffness=V_boundary_stiffness, 
#                     dt=cfg.dt,
#                     substep=cfg.substep,
#                     iteration=cfg.iteration,
#                     quasi_static=cfg.quasi_static,
#                     plane_height=cfg.ground_plane_height, 
#                     use_shape_matching=cfg.use_shape_matching,
#                     use_spring_boundary=cfg.use_spring_boundary,
#                     use_dist=True) #cfg.use_spring_boundary
#     V_ref, V_velocity_ref = step_ref.forward(softbody.V, softbody.V_velocity)
#     softbody.V = V_ref.clone()
#     softbody.V_velocity = V_velocity_ref.clone()
#     ref_V_boundary_stiffness = V_boundary_stiffness.clone()
#     energy = get_energy_boundary(softbody, softbody.V, ref_V_boundary_stiffness)
#     # set boundary energy that larger than 1e-8 as broken, 1e-5 is a eps to avoid divide 0
#     # 1e-8 is the threshold, if energy is larger than 1e-8, the first term will be 0 and the last term will be 1e-5 * 1.
#     V_boundary_stiffness = ref_V_boundary_stiffness * torch.sigmoid(1e10 * (1e-8 - energy)) + 1e-5*torch.sigmoid(1e10 * (energy - 1e-8))
#     mesh.points = softbody.V[-600:].detach().cpu().numpy()
    
#     pl.write_frame()
# pl.close()


torch.Size([3, 23])


  0%|          | 0/23 [00:00<?, ?it/s]

100%|██████████| 23/23 [00:49<00:00,  2.14s/it]
